In [17]:
import pandas as pd
import requests

In [18]:
INPUT_FILE = "./data/processed/freqs_dict.csv"

OUTPUT_FILE_PATH = "./data/reports"
OUTPUT_FILE_SUFFIX_TO_ADD = "to_add"
OUTPUT_FILE_SUFFIX_TO_UPGRADE = "to_upgrade"

VERB = "VERB"
NOUN = "NOUN"
ADJ = "ADJ"
ADV = "ADV"

POSES_TO_KEEP = {NOUN, VERB, ADJ, ADV}

LEMMA = "lemma"
POS = "pos"
FREQUENCY = "frequency"

In [19]:
df = pd.read_csv(INPUT_FILE, index_col=None, header=None, skiprows=1, names=[LEMMA, POS, FREQUENCY])

# Exploration

In [20]:
print(f"Total of {df.shape[0]} instances containing {df[LEMMA].nunique()} distinct lemmas with {df[POS].nunique()} POS(s) were loaded.\n")

display(df.groupby(POS).count())

df.sample(10)

Total of 995789 instances containing 943965 distinct lemmas with 17 POS(s) were loaded.



,lemma,frequency
pos,,
ADJ,46542,46542
ADP,790,790
ADV,19319,19319
AUX,399,399
CCONJ,136,136
DET,740,741
INTJ,30,30
NOUN,412404,412407
NUM,314404,314406


,lemma,pos,frequency
923085,Eventpartner,NOUN,1
810158,58R,PROPN,1
669228,Flügelstummel,NOUN,1
515571,Smolny-Kathedrale,NOUN,1
106937,4006,NUM,3
208350,102151,NUM,3
415839,Königskrönung,NOUN,3
223675,116788,NUM,3
479552,Steinbeis-Beratungszentrum,NOUN,2
452437,Programmaufruf,NOUN,2


## Missing values

In [21]:
df.isna().sum()

lemma        14
pos           0
frequency     0
dtype: int64

In [22]:
df[df.isna().any(axis="columns")]

,lemma,pos,frequency
5255,NaN,NUM,155
30288,NaN,NOUN,16
54021,NaN,PROPN,8
423085,NaN,PROPN,3
442342,NaN,PROPN,2
452809,NaN,PRON,2
473836,NaN,PROPN,2
490181,NaN,PROPN,2
566239,NaN,NOUN,1
617170,NaN,X,1


## Duplicate lemma-pos pairs

In [23]:
df[df.duplicated(subset=[LEMMA, POS], keep=False)].sort_values(by=[LEMMA, POS])

,lemma,pos,frequency
30288,NaN,NOUN,16
566239,NaN,NOUN,1
758022,NaN,NOUN,1
5255,NaN,NUM,155
643141,NaN,NUM,1
54021,NaN,PROPN,8
423085,NaN,PROPN,3
442342,NaN,PROPN,2
473836,NaN,PROPN,2
490181,NaN,PROPN,2


# Processing

## Missing values and duplicates

In [24]:
# Rmoving missing values will take care of duplicate lemma-pos pairs as well
df = df.dropna()
print(f"Total missing values after imputation: {df.isna().sum().sum()}")
print(f"Total duplicated lemma-pos pairs: {df[df.duplicated(subset=[LEMMA, POS])].shape[0]}")

Total missing values after imputation: 0
Total duplicated lemma-pos pairs: 0


## Invalid lemmas

In [25]:
# Removing all-caps lemmas
df = df[~df[LEMMA].str.isupper()]

# Keeping ONLY alphbetic while accepting hyphens
df = df[df[LEMMA].str.match(r'^(?=.*[A-Za-zÄÖÜäöüß])[A-Za-zÄÖÜäöüß]+(-[A-Za-zÄÖÜäöüß]+)*$')]

# Removing the leading and trailing hyphens
df[LEMMA] = df[LEMMA].str.strip("-")

df

,lemma,pos,frequency
0,der,DET,1128730
1,in,ADP,307254
2,und,CCONJ,286005
3,ein,DET,269419
4,sein,AUX,227051
...,...,...,...
995775,CIK-Zelle,NOUN,1
995776,mononukleär,ADJ,1
995777,Zzt,NOUN,1
995778,Beuth,ADJ,1


## Cleaning up POSes

In [26]:
all_poses = df.groupby(by=[POS])[LEMMA].count()
all_poses

pos
ADJ       43598
ADP         489
ADV       17784
AUX         302
CCONJ        93
DET         544
INTJ         28
NOUN     390814
NUM         313
PART         39
PRON        599
PROPN    132764
PUNCT       110
SCONJ       105
SPACE         1
VERB      21640
X         12628
Name: lemma, dtype: int64

In [27]:
df = df[df[POS].isin(POSES_TO_KEEP)]

## Cleaning up lemmas

In [28]:
df_verb = df[df[POS] == VERB]

df_verb.sort_values(FREQUENCY, ascending=False)

,lemma,pos,frequency
34,geben,VERB,25024
47,kommen,VERB,16581
53,finden,VERB,15318
57,gehen,VERB,13723
60,stehen,VERB,13375
...,...,...,...
995474,herantragen,VERB,1
995616,runtergenudeln,VERB,1
995648,vagabundieren,VERB,1
995670,vertrags,VERB,1


In [29]:
# Verb lemmas that do not end in "n"
df_verb_lemmas_not_ending_in_n = df_verb[
    ~((df_verb[LEMMA].str.endswith("n")) | (df_verb[LEMMA].str.endswith("N")))]


df_verb_lemmas_not_ending_in_n.sort_values(FREQUENCY, ascending=False)

,lemma,pos,frequency
332,lässt,VERB,2808
1383,sodass,VERB,774
1644,kannst,VERB,656
1663,schloss,VERB,646
3139,zusammengefasst,VERB,307
...,...,...,...
994684,Las,VERB,1
995078,Literaturpädagogik,VERB,1
995092,Gebraucht,VERB,1
995281,Tolerantia,VERB,1


In [30]:
df = df.drop(df_verb_lemmas_not_ending_in_n.index)

In [31]:
# Remove the duplicate POS-LEMMA pairs
not_noun_filter = df[POS] != NOUN
df.loc[not_noun_filter, LEMMA] = df.loc[not_noun_filter, LEMMA].str.lower()

df[FREQUENCY] = df.groupby([POS, LEMMA])[FREQUENCY].transform("sum")

max_freq_indexes = df.groupby([POS, LEMMA])[FREQUENCY].idxmax()
df = df.loc[max_freq_indexes].sort_values(by=[POS, LEMMA], ascending=False).reset_index(drop=True)

df

,lemma,pos,frequency
0,ühren,VERB,1
1,übrigbleiben,VERB,13
2,überzusiedeln,VERB,1
3,überzulaufen,VERB,1
4,überzugehen,VERB,10
...,...,...,...
463608,aalglatt,ADJ,3
463609,aalener,ADJ,5
463610,aalartig,ADJ,2
463611,aachenmünchener,ADJ,1


## Filtering the most frequents

In [32]:
df.groupby(POS)[LEMMA].count()

pos
ADJ      41720
ADV      16173
NOUN    390814
VERB     14906
Name: lemma, dtype: int64

In [36]:
TOP_PERCENTILES = {
    VERB: 98,
    NOUN: 99.9,
    ADJ: 99,
    ADV: 99,
}

df_top = pd.DataFrame(columns=df.columns)
for pos in POSES_TO_KEEP:
    df_pos = df[df[POS] == pos]

    percentile = TOP_PERCENTILES[pos]
    top_quantile = df_pos[FREQUENCY].quantile(percentile/100)
    df_pos = df_pos[df_pos[FREQUENCY] >= top_quantile]

    df_top = pd.concat([df_top, df_pos])
    
for pos in POSES_TO_KEEP:
    print(f"{pos} -> {len(df_top[df_top[POS] == pos])}")
    
df_top

ADJ -> 418
ADV -> 163
NOUN -> 392
VERB -> 299


,lemma,pos,frequency
421898,übrig,ADJ,660
421901,üblich,ADJ,466
422247,östlich,ADJ,501
422258,österreichisch,ADJ,570
422268,örtlich,ADJ,504
...,...,...,...
14137,angeben,VERB,829
14221,anbieten,VERB,1982
14327,achten,VERB,744
14529,absolvieren,VERB,982


# Anki analysis

## Fetching current notes from Anki

In [38]:
# NOTE: Before executing this cell, Anki should be running along with AnkiConnect addon active.

MODIFIER = "MODIFIER"
DEUTSCH = "Deutsch"
IS_HIGHFREQUENCY = "is_highfrequency"
IS_HIGHFREQUENCY_VALUE = "Yes"

ANKI_URL = "http://127.0.0.1:8765"
ANKI_VERB = "My-German-Verb"
ANKI_NOUN = "My-German-Noun"
ANKI_MODIFIER = "My-German-Modifier"
ANKI_POSES = [ANKI_VERB, ANKI_NOUN, ANKI_MODIFIER]
ANKI_QUERY = " OR ".join([f"note:{notetype}" for notetype in ANKI_POSES])

def extract_main_word(word: str) -> str:
    """Ignores the optional prefix and optional suffix and extracts the main word"""
    word_parts = word.split()
    return word_parts[len(word_parts) // 2]

# Retreieving note IDs from Anki
note_ids = requests.post(
    ANKI_URL, timeout=(0.5, 3), json={"action": "findNotes", "version": 6, "params": {"query": ANKI_QUERY}}
).json()["result"]

# Retreieving note info from Anki
note_info = requests.post(
    ANKI_URL, timeout=(0.5, 3), json={"action": "notesInfo", "version": 6, "params": {"notes": note_ids}}
).json()["result"]

anki_notetypes = []
anki_deutschs = []
anki_lemmas = []
anki_is_highfrequency = []
for note in note_info:
    anki_notetypes.append(note["modelName"])
    word = note["fields"][DEUTSCH]["value"]
    anki_deutschs.append(word)
    anki_lemmas.append(extract_main_word(word))
    anki_is_highfrequency.append(note["fields"][IS_HIGHFREQUENCY]["value"])

df_anki = pd.DataFrame(
    {
        DEUTSCH: anki_deutschs,
        LEMMA: anki_lemmas,
        POS: anki_notetypes,
        IS_HIGHFREQUENCY: anki_is_highfrequency,
    }
)

df_anki

,Deutsch,lemma,pos,is_highfrequency
0,der Familienname,Familienname,My-German-Noun,Yes
1,die Frage,Frage,My-German-Noun,Yes
2,der Nachname,Nachname,My-German-Noun,
3,der Name,Name,My-German-Noun,Yes
4,der Vorname,Vorname,My-German-Noun,
...,...,...,...,...
2868,zwingend,zwingend,My-German-Modifier,Yes
2869,äußerst,äußerst,My-German-Modifier,Yes
2870,öfter,öfter,My-German-Modifier,Yes
2871,überraschend,überraschend,My-German-Modifier,Yes


## Comparing Anki notes with high-frequency words

In [39]:
# Notes that already exist in Anki and should be flagged as high frequency
df_to_upgrade = pd.DataFrame(columns=df_anki.columns)

# High frequency words that are absent in Anki
df_to_add = pd.DataFrame(columns=df_top.columns)

# Mappings between spaCy POSes and Anki POSes
mappings = {
    VERB: ANKI_VERB,
    NOUN: ANKI_NOUN,
    ADJ: ANKI_MODIFIER,
    ADV: ANKI_MODIFIER
}

In [40]:
for pos in POSES_TO_KEEP:
    anki_pos = mappings[pos]
    
    df_anki_pos = df_anki[df_anki[POS] == anki_pos]
    df_top_pos = df_top[df_top[POS] == pos]

    anki_pos_lemmas = df_anki_pos[LEMMA].unique()
    df_pos_to_add = df_top_pos[~df_top_pos[LEMMA].isin(anki_pos_lemmas)]

    top_pos_lemmas = df_top_pos[LEMMA].unique()
    df_pos_to_upgrade = df_anki_pos[(df_anki_pos[LEMMA].isin(top_pos_lemmas)) & (df_anki_pos[IS_HIGHFREQUENCY] != "Yes")]
    
    df_to_add = pd.concat([df_to_add, df_pos_to_add], ignore_index=True)
    df_to_upgrade = pd.concat([df_to_upgrade, df_pos_to_upgrade], ignore_index=True)
    
    print(f"total {pos}: {len(df_top_pos)}")
    print(f"total {anki_pos}: {len(df_anki_pos)}")
    print(f"total {pos} to add: {len(df_pos_to_add)}")
    print(f"total {anki_pos} to upgrade: {len(df_pos_to_upgrade)}\n")


total ADJ: 418
total My-German-Modifier: 952
total ADJ to add: 110
total My-German-Modifier to upgrade: 0

total ADV: 163
total My-German-Modifier: 952
total ADV to add: 22
total My-German-Modifier to upgrade: 0

total NOUN: 392
total My-German-Noun: 1403
total NOUN to add: 135
total My-German-Noun to upgrade: 0

total VERB: 299
total My-German-Verb: 518
total VERB to add: 117
total My-German-Verb to upgrade: 0



## Saving the results

In [41]:
df_to_upgrade[IS_HIGHFREQUENCY] = IS_HIGHFREQUENCY_VALUE

for pos in POSES_TO_KEEP:
    df_to_save = df_to_add[df_to_add[POS] == pos].drop([POS, FREQUENCY], axis="columns")
    if not len(df_to_save): continue
    df_to_save.to_csv(f"{OUTPUT_FILE_PATH}/{pos}_{OUTPUT_FILE_SUFFIX_TO_ADD}.csv", index=False, header=False)

for anki_pos in ANKI_POSES:
    df_to_save = df_to_upgrade[df_to_upgrade[POS] == anki_pos].drop([POS, LEMMA], axis="columns")
    if not len(df_to_save): continue
    df_to_save.to_csv(f"{OUTPUT_FILE_PATH}/{anki_pos}_{OUTPUT_FILE_SUFFIX_TO_UPGRADE}.csv", index=False, header=False)